# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
import gradio as gr
from anthropic import Anthropic


In [2]:
# Initialization

load_dotenv(override=True)

MODEL = "claude-3-haiku-20240307"
claude_key = os.getenv('CLAUDE_API_KEY')
anthropic_client = Anthropic(api_key=claude_key)


In [12]:
system_message = "You are a concise, helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. Respond only with the final answer, no explanations, and do not show your reasoning steps or thinking. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "You have access to tools, but only use them when necessary. If a tool is not required, respond as normal"

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [4]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [86]:
get_ticket_price("London")  # Example call to test the function

Tool get_ticket_price called for London


'$799'

In [5]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "input_schema": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [6]:
# And this is included in a list of tools:

tools = [price_function]

## Getting Anthropic to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [52]:
import json

def chat(message, history):
    
    history = [{k: v for k, v in d.items() if k not in ("metadata", "options")} for d in history]
    messages = history + [{"role": "user", "content": message}]
    response = anthropic_client.messages.create(model=MODEL, max_tokens = 1000,system=system_message, messages=messages, tools=tools)
    messages.append(
        {"role": "assistant", "content": response.content}
    )  # Append the original message to maintain context
    # Check if the response indicates a tool use
     # If the response indicates a tool use, handle it  
    
    if response.stop_reason == "tool_use":
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
       # message = response.content[0].text
        result = handle_tool_call(tool_name, tool_input)
       
        # Append the tool result to the messages
        messages.append(
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id":tool_use.id,
                        "content": str(result),
                    },
                ],
            }
        )
        response = anthropic_client.messages.create(model=MODEL, max_tokens = 1000,system=system_message, messages=messages)
    return response.content[0].text
    

In [53]:
# We have to write that function handle_tool_call:

def handle_tool_call(name, input):
    if (name == "get_ticket_price"):
        city = input["destination_city"]
        price = get_ticket_price(city)
    else:
        price = "Unknown"
    return json.dumps({"destination_city": city,"price": price})

In [54]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7918
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Tokyo
Tool get_ticket_price called for London
Tool get_ticket_price called for Jamaica
